In [1]:
import sys
import scanpy as sc
import pandas as pd
from cnmf import cNMF
import numpy as np
import tqdm as tqdm
import gseapy as gp
from tqdm.auto import tqdm

In [2]:
save_path = 'p22_data/'

In [3]:
'''LOAD KOENIG'''

'''
de_file = 'data/Koenig_DE.csv'
l2gc = 'log2FoldChange'
padj = 'padj'
data_name = 'koenig'
'''

"\nde_file = 'data/Koenig_DE.csv'\nl2gc = 'log2FoldChange'\npadj = 'padj'\ndata_name = 'koenig'\n"

In [4]:
'''LOAD CHAFFIN BENDER'''
'''
de_file = 'data/chaffin_DE.csv'
l2gc = 'CellBender:\r\nlogFC'
padj = 'CellBender:\r\nAdjusted P-Value'
data_name = 'bender'
'''

'''LOAD CHAFFIN RANGER'''

de_file = 'data/chaffin_DE.csv'
l2gc = 'CellRanger:\r\nlogFC'
padj = 'CellRanger:\r\nAdjusted P-Value'
data_name = 'ranger'


In [5]:
all_de = pd.read_csv(de_file,index_col =0) 
p5_all_de = all_de[all_de[padj] <= 0.05].sort_values(padj, ascending = True).index.tolist()
p5_up_de = all_de[(all_de[padj] <= 0.05) & (all_de[l2gc] > 0)].sort_values(l2gc, ascending = False).index.tolist()
p5_down_de = all_de[(all_de[padj] <= 0.05) & (all_de[l2gc] < 0)].sort_values(l2gc, ascending = True).index.tolist()

In [6]:
g_de_sets = {}

g_de_sets['p<0.05_genes'] = p5_all_de[:500]
g_de_sets['upregulated_genes']= p5_up_de[:500]
g_de_sets['downregulated_genes'] = p5_down_de[:500]

In [7]:
o_de_sets= {}

#o_de_sets['all_genes'] = all_de.index.tolist()
o_de_sets['p<0.05_genes'] = p5_all_de
o_de_sets['upregulated_genes']= p5_up_de
o_de_sets['downregulated_genes'] = p5_down_de

In [8]:
up, down = 0,0

for v1 in g_de_sets['p<0.05_genes']:
    if v1 in g_de_sets['upregulated_genes']:
        up += 1
    if v1 in g_de_sets['downregulated_genes']:
        down += 1


print(up)
print(down)

174
128


In [ ]:
cnmf_obj = cNMF(output_dir="ml_program/adatafinal22", name="k22run")
cnmf_obj.consensus(k=22, density_threshold=0.1)
usage, spectra_scores, spectra_tpm, top_genes = cnmf_obj.load_results(K=22, density_threshold=0.1)

/hpc/home/zy231/.local/lib/python3.11/site-packages/scanpy/preprocessing/_scale.py:309: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [ ]:
nmf_ranks = {}

for program_id in spectra_scores.columns:
    rnk = spectra_scores[program_id]
    rnk = rnk.sort_values(ascending=False)
    rnk.index = rnk.index.str.upper()
    
    nmf_ranks[program_id] = rnk

print(f"Created ranked lists for {len(nmf_ranks)} programs.")

In [ ]:
nmf_list = {}
top = 500

for program_id in spectra_scores.columns:
    rnk = spectra_scores[program_id]
    rnk = rnk.sort_values(ascending = False).head(top)
    genes = rnk.index.str.upper().tolist()
    
    nmf_list[program_id] = genes

In [ ]:
g_all_results = {}

for i, ranks in nmf_ranks.items():
    
    prerank_results = gp.prerank(
        rnk=ranks,                        # Your DE list
        gene_sets=g_de_sets,    # Your cNMF Top 50 Lists
        threads=40,
        min_size=5,                     # Low min_size is crucial for custom sets
        permutation_num=1000,
        #outdir='program_analysis_results/koenig_gsea_results',     # Will save plots to this folder
        seed=14,
    )

    if not prerank_results.res2d.empty:
        print(f"Program {i} - Top hit: {prerank_results.res2d.iloc[0]['Term']}")
        g_all_results[i] = prerank_results.res2d
    else:
        print(f"Program {i} - No significant pathways found.")

In [ ]:
g_pathways_list = []

for i, result in g_all_results.items():
    if result.empty:
        continue

    sorted_df = result.sort_values(by = 'NES', ascending =False)

    sig_df = sorted_df[sorted_df['FDR q-val'] <= 0.05]
    #sig_df = sorted_df

    sig_df.insert(0,'Program_ID', i)

    g_pathways_list.append(sig_df)

g_paths_sum = pd.concat(g_pathways_list, ignore_index =True)
g_paths_sum.to_csv(save_path + data_name + '_de_gsea.csv', index = False)
    

In [ ]:
o_all_results = {}

for i, genes in nmf_list.items():
    
    enr_results = gp.enrichr(
        gene_list=genes,
        gene_sets=o_de_sets,
        cutoff=0.05,  # Only keep results with P-value < 0.05
        #background = all_de.index.tolist(),
        #verbose = True
    )
    if enr_results.res2d is not None:
        print(f"Program {i} - Top hit: {enr_results.res2d.sort_values('Adjusted P-value').iloc[0]['Term']}")
        o_all_results[i] = enr_results.res2d
    else:
        print(f"Program {i} - No significant pathways found.")



In [ ]:
o_pathways_list = []

for i, result in o_all_results.items():
    if result.empty:
        continue

    sorted_df = result.sort_values(by='Adjusted P-value', ascending=True)
    sig_df = sorted_df[sorted_df['Adjusted P-value'] <= 0.05].copy()

    if not sig_df.empty:
        sig_df.insert(0, 'Program_ID', i)
        o_pathways_list.append(sig_df)

if o_pathways_list:
    o_paths_sum = pd.concat(o_pathways_list, ignore_index=True)
    o_paths_sum.to_csv(save_path + data_name + '_de_ora.csv', index=False)
    print(f"Saved aggregated results with {len(o_paths_sum)} rows.")
else:
    print("No significant results found to save.")